## Example for compositing other fields over/beneath tracked eddies (daily data)

#### High pass filter other fields (already on 0.25deg)

In [1]:
from py_eddy_tracker.dataset.grid import RegularGridDataset
from datetime import datetime, timedelta
import numpy as np


In [3]:
varname='to'
yyyy=2002 #up to 2008

print('High pass filter daily '+varname+' for year='+str(yyyy))

expid='erc1011'
fq='dm'
wavelength=700
daterng=str(yyyy)+'0101-'+str(yyyy)+'1231'
datearr=np.arange(datetime(yyyy,1,1), datetime(yyyy+1,1,1), timedelta(days=1)).astype(datetime)

datadir='reg25/'+expid+'/'+varname+'/'
outdir='reg25/'+expid+'/eddytrack/'+varname+'/sm'+str(int(wavelength))+'km/'
#Must create dirs first. 

if varname=='to' or varname=='so' or varname=='rho':
    zidx=1
    varfile=datadir+expid+'_'+varname+'_'+str(zidx)+'_'+fq+'_'+daterng+'_IFS25.nc'
else:
    varfile=datadir+expid+'_'+varname+'_'+fq+'_'+daterng+'_IFS25.nc'


High pass filter daily to for year=2002


In [4]:
def besselhighpass(varfile, varname, datearr, tt):
    wavelength=700  #choice of spatial cutoff for high pass filter in km
    step_ht=0.005 #intervals to search for closed contours (5mm in this case)
    g = RegularGridDataset(varfile, "lon", "lat", centered=True, indexs = dict(time=tt))
    g.dimensions['time']=1  #extracts only one time step that was specified by indexs = dict(time=tt)
    if varname=='rho':
        g.bessel_high_filter('rhopoto', wavelength, order=1)
    else:
        g.bessel_high_filter(varname, wavelength, order=1) #perfroms only on 1 time index
    date = datearr[tt] # detect each timestep individually because of memory issues
    if varname=='to' or varname=='so' or varname=='rho':
        zidx=1
        g.write(outdir+expid+'_'+varname+'_'+str(zidx)+'_'+fq+'_'+date.strftime('%Y%m%d')+'_IFS25_hp'+str(wavelength)+'.nc')
    else:
        g.write(outdir+expid+'_'+varname+'_'+fq+'_'+date.strftime('%Y%m%d')+'_IFS25_hp'+str(int(wavelength))+'.nc')


In [ ]:
for tt in range(0,len(datearr)):
    date = datearr[tt]
    print('High pass filter of '+varname+' for '+date.strftime('%Y%m%d'))
    besselhighpass(varfile, varname, datearr, tt)


#### Extract and build eddy composites of the other fields 
Along the identified tracks, gather 2.5x2.5deg around eddy center and save. 

In [5]:
import os 
import shutil
import numpy as np
from datetime import datetime, timedelta
import xarray as xr


In [7]:
varname='to'

expid='erc1011'
fq='dm'
eddydir='reg25/'+expid+'/eddytrack/ssh/'+fq+'/'
tracker_dir=eddydir+'tracks/'

# eddy_type='cyclonic'
eddy_type='anticyclonic'

#dscorres = xr.open_dataset(tracker_dir+expid+'_'+eddy_type+'_'+fq+'_correspondance.nc')
dstracks = xr.open_dataset(tracker_dir+eddy_type+'_tracks.nc')
#dsshort = xr.open_dataset(tracker_dir+eddy_type+'_short.nc')
#dsuntrack = xr.open_dataset(tracker_dir+eddy_type+'_untracked.nc')

wavelength=700
dlon=2.5
dlat=2.5
res=0.25
npts=int(dlon/res) #number of points from centre


In [8]:
#Get desired region [Agulhas leakage in this case]
ARidx = np.argwhere((dstracks.latitude.values<=-30) & (dstracks.latitude.values>=-45) &
                     (dstracks.longitude.values>0) & (dstracks.longitude.values<25))

#Get track IDs for Agulhas rings, remove all duplicates
ARtrackid=np.array(sorted(list(set(dstracks.track.values[ARidx].squeeze()))))
print('Track IDs=',ARtrackid)

#Get number of obs for each track for Agulhas rings
trackIDs=dstracks.track.values
tracklen=[]
for ii in range(trackIDs.max()+1):
    tracklen.append(len(np.argwhere(trackIDs == ii)))
lentrack=np.array(tracklen)[ARtrackid]
print('No. of obs for each tracked ID =',lentrack)

#Remove tracks with less than 60 days
newARtrackid=np.delete(ARtrackid,np.r_[np.argwhere(lentrack<60)])
print('Track IDs that last more than 60 days=',newARtrackid)
lentrack=np.array(tracklen)[newARtrackid]
print('No. of obs for each tracked ID =',lentrack)
del(tracklen)


Track IDs= [    4    29   125   127   133   171   176   179   219   263   280   286
   351   407   419   432   461   471   525   575   582   589   756   771
   793   811   828   870   976  1007  1024  1070  1104  1152  1161  1191
  1200  1212  1251  1314  1319  1321  1391  1392  1465  1520  1534  1535
  1725  1741  1752  1909  1913  1923  1935  2072  2101  2102  2113  2122
  2131  2134  2146  2231  2234  2277  2300  2328  2412  2443  2467  2563
  2567  2652  2717  2743  2863  2889  2944  2979  3020  3035  3135  3294
  3296  3297  3317  3337  3356  3421  3438  3472  3501  3521  3537  3621
  3666  3714  3730  3827  3854  3904  3933  3978  4007  4098  4153  4170
  4194  4202  4239  4323  4399  4404  4481  4504  4536  4621  4669  4670
  4692  4715  4785  4793  4802  4809  4836  4857  4898  4905  5088  5136
  5205  5208  5211  5250  5354  5421  5461  5463  5503  5557  5687  5708
  5713  5734  5760  6026  6049  6078  6148  6304  6311  6367  6422  6431
  6432  6476  6563  6630  6642  6776  67

In [9]:
def geteddy_alongtrack(dmvarfile,varname,loncen,latcen,dlon=2.5,dlat=2.5):
    dsvar=xr.open_dataset(dmvarfile)
    if varname=='rho':
        FIELD=dsvar['rhopoto']
    else:
        FIELD=dsvar[varname]
    lonmin=loncen-dlon
    lonmax=loncen+dlon
    latmin=latcen-dlat
    latmax=latcen+dlat
    if (lonmin < 0) & (lonmax < 0):
        FIELDcomp=FIELD.sel(lon=slice(lonmin+360,lonmax+360),lat=slice(latmin,latmax))
    elif (lonmin < 0) & (lonmax >= 0):
        FIELDcomp=xr.concat([FIELD.sel(lon=slice(lonmin+360,360),lat=slice(latmin,latmax)),FIELD.sel(lon=slice(0,lonmax),lat=slice(latmin,latmax))],dim='lon')
    elif (lonmax > 360):
        FIELDcomp=xr.concat([FIELD.sel(lon=slice(lonmin,360),lat=slice(latmin,latmax)),FIELD.sel(lon=slice(0,lonmax-360),lat=slice(latmin,latmax))],dim='lon')
    else:
        FIELDcomp=FIELD.sel(lon=slice(lonmin,lonmax),lat=slice(latmin,latmax))
    return FIELDcomp


#Note that this extract eddy tracks but does not normalise to eddy radius
def extract_eddytrack_raw(dstracks,tridx,varname='ssh',expid='erc1011',fq='dm',wavelength=700,dlon=2.5,dlat=2.5,res=0.25):
    npts=int(dlon/res)
    alongtrackidx=np.argwhere(dstracks.track.values==tridx)
    Reff=dstracks.effective_radius.values[alongtrackidx] #in metres
    loncen=dstracks.longitude.values[alongtrackidx]
    latcen=dstracks.latitude.values[alongtrackidx]
    timearr=dstracks.time.values[alongtrackidx].flatten()
    date_arr=[]
    FIELDcomp=[]
    for tt in range(len(timearr)):
        date_arr.append(datetime.strptime(str(timearr[tt])[:10], '%Y-%m-%d'))
        date=date_arr[tt]
        # print('Extracting for '+date.strftime('%Y%m%d'))
        #Get high pass filtered classified data
        outdir='reg25/'+expid+'/eddytrack/'
        if varname=='rho' or varname=='to':
            dmvarfile=outdir+varname+'/sm'+str(int(wavelength))+'km/'+expid+'_'+varname+'_1_'+fq+'_'+date.strftime('%Y%m%d')+'_IFS25_hp'+str(wavelength)+'.nc'
        else:
            dmvarfile=outdir+varname+'/sm'+str(int(wavelength))+'km/'+expid+'_'+varname+'_'+fq+'_'+date.strftime('%Y%m%d')+'_IFS25_hp'+str(wavelength)+'.nc'
        FIELDcomp.append(geteddy_alongtrack(dmvarfile,varname,loncen[tt][0],latcen[tt][0],dlon=dlon,dlat=dlat))
        del(dmvarfile)

    for ii in range(len(FIELDcomp)):
        FIELDcomp[ii]=FIELDcomp[ii].assign_coords(lat=np.arange(-npts,npts)*res,lon=np.arange(-npts,npts)*res)
    
    return alongtrackidx, date_arr, FIELDcomp, FIELDcomp[0].attrs

#Note it is not normalized to eddy radius
def appendto_eddycomp_dataset(dsSSH,varname,FIELDcomp,date_arr,npts,res,FIELDattrs):
    dsSSH=dsSSH.merge(xr.DataArray(data=np.array(FIELDcomp).squeeze(),
                 coords={'time':date_arr, 'y':np.arange(-npts,npts)*res, 'x':np.arange(-npts,npts)*res},
                 dims=['time','y','x'],name=varname,attrs=FIELDattrs))
    return dsSSH



In [ ]:
for tridx in newARtrackid:
    print('Extracting for eddy track ',tridx)
    #Extract tracked eddies
    alongtrackidx, date_arr, FIELDcomp, FIELDattrs = extract_eddytrack_raw(dstracks,tridx,varname=varname,wavelength=wavelength,dlon=dlon,dlat=dlat,res=res)
    
    #Load composite SSH dataset
    fileout=tracker_dir+expid+'_'+eddy_type+'_'+str(dlon)+'x'+str(dlat)+'deg_trackID_'+str(tridx)+'.nc'
    tempout=tracker_dir+expid+'_'+eddy_type+'_'+str(dlon)+'x'+str(dlat)+'deg_trackID_'+str(tridx)+'_temp.nc'
    dsSSH=xr.open_dataset(fileout)
    #Append and save dataset
    dsnew=appendto_eddycomp_dataset(dsSSH,varname,FIELDcomp,date_arr,npts,res,FIELDattrs)
    dsnew.to_netcdf(tempout)
    shutil.move(tempout,fileout)
